# Querying Elasticsearch

This simple example demonstrates how you can retrieve and query previous requests and responses thanks to Seldon Deploy's Elasticsearch capabilities. 

### Pre-Requisites

If you would like to follow along closely, then you will need to deploy the Scikit-Learn Iris model which Seldon provides as an example.

Consequently, you can pull the model from its bucket by using this URI:
```
gs://seldon-models/sklearn/iris
```
It can be deployed out using the pre-packaged Scikit-Learn inference server. 

Once you have deployed the iris model onto your Seldon Deploy cluster, then send a couple of requests to it using the following sample data: 
```
{
    "data": {
	"names": ["Sepal length","Sepal width","Petal length", "Petal Width"],
	"ndarray": [
	    [6.8,  2.8,  4.8,  1.4],
	    [6.0,  3.4,  4.5,  1.6]
	]
    }
}

```

### Connecting to Elasticsearch

First need to setup port forwarding to localhost in order to easily call the Elasticsearch instance. If you have `kubectl` access to your Seldon Deploy cluster this can be achieved by running the following command in your terminal window:
```
kubectl port-forward -n seldon-logs svc/elasticsearch-opendistro-es-client-service 9200:9200
```

If you're successful you will see the following output:
```
Forwarding from 127.0.0.1:9200 -> 9200
Forwarding from [::1]:9200 -> 9200
```

We can then import the relevant `elasticsearch` module. We do not need to specify the URI and port of the Elasticsearch instance we would like to communicate with. The default value is `localhost:9200` which is where our port forward is pointing to. 

NOTE: The `elasticsearch` Python module version should match the version of Elasticsearch being run by Seldon Deploy. e.g. Seldon Deploy v1.0 uses Elasticsearch 7.6.0, therefore the Python module which you install should be `elasticsearch>=7.0.0,<8.0.0` 

In [38]:
from elasticsearch import Elasticsearch
from elasticsearch.client import CatClient

client = Elasticsearch()

### Listing the Available Indices
Each new model deployment generates a new Elasticsearch index. All of the requests and responses for this deployment are then stored in this newly created index. 

The model deployment index names adhere to the following pattern: 
```
inference-log-seldon-<NAMESPACE>-<DEPLOYMENT_NAME>-<PREDICTOR_TYPE>
```
e.g. If we created a new deployment called `elastic-test` in the `development` namespace, and cared about the requests and responses which our `default` predictor had served the index of interest would be:
```
inference-log-seldon-development-elastic-test-default
```

If we are not sure about the indices we have available we can use the `CatClient` class.

In [21]:
indices = CatClient.indices(client, format='json')
indices

[{'health': 'green',
  'status': 'open',
  'index': 'inference-log-seldon-development-income-default',
  'uuid': 'WROy05hSQp2HYl8_4nRREg',
  'pri': '1',
  'rep': '1',
  'docs.count': '803',
  'docs.deleted': '803',
  'store.size': '20.8mb',
  'pri.store.size': '10.4mb'},
 {'health': 'green',
  'status': 'open',
  'index': 'inference-log-seldon-development-liveperson-test-default',
  'uuid': 'M8BHz70GSxmjJe9YH4Lz9g',
  'pri': '1',
  'rep': '1',
  'docs.count': '3505',
  'docs.deleted': '267',
  'store.size': '3.3mb',
  'pri.store.size': '1.7mb'},
 {'health': 'green',
  'status': 'open',
  'index': '.kibana_1',
  'uuid': '4Hw-FWfYRbiNKqm4HuwigQ',
  'pri': '1',
  'rep': '1',
  'docs.count': '1',
  'docs.deleted': '0',
  'store.size': '7.4kb',
  'pri.store.size': '3.7kb'},
 {'health': 'green',
  'status': 'open',
  'index': 'inference-log-seldon-development-elastic-test-default',
  'uuid': 'gAB1VpqeSE-kDEahI1_Hxg',
  'pri': '1',
  'rep': '1',
  'docs.count': '2',
  'docs.deleted': '0',
  '

Once we have the index we care about identified we can now view all of the documents stored within the index. 

(Results are paginated with only 10 documents showing up at a time, so if you were expecting more to be displayed that may be why)

In [43]:
resp = client.search(index="inference-log-seldon-development-elastic-test-default")
resp

{'took': 2,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 2, 'relation': 'eq'},
  'max_score': 1.0,
  'hits': [{'_index': 'inference-log-seldon-development-elastic-test-default',
    '_type': 'inferencerequest',
    '_id': '8504396c-dd5f-48a3-9ff5-44b1ce3fa0d6-item-0',
    '_score': 1.0,
    '_source': {'response': {'payload': {'data': {'names': ['t:0',
         't:1',
         't:2'],
        'ndarray': [[0.008074020139120223,
          0.7781601484223128,
          0.21376583143856684],
         [0.04192755260928376, 0.4765961786674123, 0.481476268723304]]},
       'meta': {'requestPath': {'elastic-test-container': 'seldonio/sklearnserver:1.5.0'}}},
      'dataType': 'tabular',
      'elements': {'t:0': [0.04192755260928376],
       't:1': [0.4765961786674123],
       't:2': [0.481476268723304]},
      'instance': [0.008074020139120223,
       0.7781601484223128,
       0.21376583143856684],
      'names': ['t:

Elasticsearch stores all documents which match the search under the `hits` key. With details of each of those documents in the nested `hits` array. 

Seldon captures metadata alongside the request and response of the model within this `hits` array. We can query for specific requests using the `request` and `elements` dictionary which Deploy automatically configures for us. This makes matching on certain data values super simple, as demonstrated below. 

In [56]:
query = client.search(index="inference-log-seldon-development-elastic-test-default",
                      body={
                          "query": {
                              "match": {
                                  "request.elements.Petal length": 4.5
                              }
                          }
                      })
query

{'took': 3,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 2, 'relation': 'eq'},
  'max_score': 1.0,
  'hits': [{'_index': 'inference-log-seldon-development-elastic-test-default',
    '_type': 'inferencerequest',
    '_id': '8504396c-dd5f-48a3-9ff5-44b1ce3fa0d6-item-0',
    '_score': 1.0,
    '_source': {'response': {'payload': {'data': {'names': ['t:0',
         't:1',
         't:2'],
        'ndarray': [[0.008074020139120223,
          0.7781601484223128,
          0.21376583143856684],
         [0.04192755260928376, 0.4765961786674123, 0.481476268723304]]},
       'meta': {'requestPath': {'elastic-test-container': 'seldonio/sklearnserver:1.5.0'}}},
      'dataType': 'tabular',
      'elements': {'t:0': [0.04192755260928376],
       't:1': [0.4765961786674123],
       't:2': [0.481476268723304]},
      'instance': [0.008074020139120223,
       0.7781601484223128,
       0.21376583143856684],
      'names': ['t: